#### Importing the libraries 

In [7]:
!conda install -c conda-forge folium=0.5.0 --yes

Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /home/nbuser/anaconda3_420:

The following NEW packages will be INSTALLED:

    _libgcc_mutex: 0.1-main                    
    altair:        2.2.2-py35_1     conda-forge
    branca:        0.3.1-py_0       conda-forge
    folium:        0.5.0-py_0       conda-forge
    readline:      7.0-ha6073c6_4              
    typing:        3.6.6-py35_0     conda-forge
    vincent:       0.4.4-py_1       conda-forge

The following packages will be UPDATED:

    conda:         4.3.31-py35_0                --> 4.5.11-py35_0        conda-forge
    pycosat:       0.6.1-py35_1                 --> 0.6.3-py35h470a237_1 conda-forge

The following packages will be SUPERSEDED by a higher-priority channel:

    conda-env:     2.6.0-h36134e3_1             --> 2.6.0-1              conda-forge

_libgcc_mutex- 100% |################################| Time: 0:00:00 300.59 kB/s
conda-env-2.6. 

In [4]:
import pandas as pd
import folium
import requests
from pandas.io.json import json_normalize

#### Reading the data from the US Census bureau

In [5]:
us = pd.read_csv('US_Counties_with_FIPS.csv')

#### Extracting only New York data

In [6]:
ny = us.loc[us['state_abbr'] == 'NY']

In [7]:
ny.head()

,state_abbr,geoid,name,latitude,longitude
1828,NY,36001,Albany County,42.588240,-73.974010
1829,NY,36003,Allegany County,42.247853,-78.026153
1830,NY,36005,Bronx County,40.848711,-73.852939
1831,NY,36007,Broome County,42.161977,-75.830283
1832,NY,36009,Cattaraugus County,42.239099,-78.662332


#### Establishing a connection with the Foursqure API

In [1]:
CLIENT_ID = '<Removed>'
CLIENT_SECRET = '<Removed>'
VERSION = "20190520"

#### Defining the parameters to the API

In [11]:
search_query = 'Car Wash'
radius = 10000
LIMIT = 1000

#### Create an empty data frame to store the data

In [8]:
df = pd.DataFrame(columns= ['name','location.lat','location.lng'])

In [9]:
df

,name,location.lat,location.lng


#### Query the Foursquare API for Car Washes in New York State

In [15]:
for lat,lng in zip(ny['latitude'], ny['longitude']):
    url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, lat, lng, VERSION, search_query, radius, LIMIT)
    results = requests.get(url).json()
    venues = results['response']['venues']
    print(len(venues))
    if len(venues) >0:
        dataframe = json_normalize(venues)
        df1 = dataframe[['name','location.lat','location.lng']]
        df = pd.concat([df, df1],  ignore_index=True)

12
7
50
50
9
50
6
50
15
2
12
38
6
42
50
0
0
21
50
11
0
0
46
50
6
18
4
0
9
50
50
0
31
50
42
50
0
9
13
50
50
20
50
50
3
50
50
16
12
1
10
50
33
23
50
3
8
15
0
50
7
13


In [16]:
df.head()

,name,location.lat,location.lng
0,Advanced Car Wash,42.653310,-73.926680
1,The Employer Alliance for Affordable Health Care,42.613623,-73.973587
2,Office Ala Carte,42.614497,-73.972085
3,Carl's Hearth Services,42.578463,-73.880506
4,Sounds And Motion Car Audio,42.656670,-73.928864


#### Generate a Heat Map

In [17]:
from folium import plugins
from folium.plugins import HeatMap


map_hooray = folium.Map(location=[41.937734, -74.662139],
                    zoom_start = 7) 

heat_df = df[['location.lat', 'location.lng']]
heat_df = heat_df.dropna(axis=0, subset=['location.lat','location.lng'])


heat_data = [[row['location.lat'],row['location.lng']] for index, row in heat_df.iterrows()]

HeatMap(heat_data).add_to(map_hooray)

map_hooray

#### Zoom in Long Island

In [21]:
# Long Island
from folium import plugins
from folium.plugins import HeatMap


map_hooray = folium.Map(location=[40.7891, -73.1350],
                    zoom_start = 10) 

heat_df = df[['location.lat', 'location.lng']]
heat_df = heat_df.dropna(axis=0, subset=['location.lat','location.lng'])


heat_data = [[row['location.lat'],row['location.lng']] for index, row in heat_df.iterrows()]

HeatMap(heat_data).add_to(map_hooray)

map_hooray

#### As you can see that in Long Island there is quite room for some Car Washes